# Column Analysis of The Video Game Sales
In this notebook, we examine the correlations and the impact of each column on the outcome.

<h3>Importing libraries</h3>

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

<h3>Loading the dataset</h3>

In [2]:
df_sales = pd.read_csv("./data/vgsales.csv")

In [3]:
df_sales.count()

Rank            16598
Name            16598
Platform        16598
Year            16327
Genre           16598
Publisher       16540
NA_Sales        16598
EU_Sales        16598
JP_Sales        16598
Other_Sales     16598
Global_Sales    16598
dtype: int64

<h3>Dropping Rows with NaNs</h3>

In [4]:
df_sales.isna().sum()

Rank              0
Name              0
Platform          0
Year            271
Genre             0
Publisher        58
NA_Sales          0
EU_Sales          0
JP_Sales          0
Other_Sales       0
Global_Sales      0
dtype: int64

In [5]:
df_sales = df_sales.dropna()

In [6]:
df_sales.isna().sum()

Rank            0
Name            0
Platform        0
Year            0
Genre           0
Publisher       0
NA_Sales        0
EU_Sales        0
JP_Sales        0
Other_Sales     0
Global_Sales    0
dtype: int64

<h3>Dividing columns into Continous & Categorical features</h3>

In [7]:
continuous_feats = df_sales.select_dtypes(include=['float64', 'int64']).columns

In [8]:
continuous_feats

Index(['Rank', 'Year', 'NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales',
       'Global_Sales'],
      dtype='object')

In [9]:
categorical_feats = df_sales.select_dtypes(include=['object', 'category']).columns

In [10]:
categorical_feats

Index(['Name', 'Platform', 'Genre', 'Publisher'], dtype='object')

<h3>Correlation Coefficient</h3>
It is used, when you have numerical features and want to find linear relationships between features and the target variable.

In [11]:
df_sales[continuous_feats].corr()["Global_Sales"].sort_values(ascending=False)

Global_Sales    1.000000
NA_Sales        0.941269
EU_Sales        0.903264
Other_Sales     0.747964
JP_Sales        0.612774
Year           -0.074647
Rank           -0.426975
Name: Global_Sales, dtype: float64

***Based on the results of the correlation coefficient analysis, we have chosen to remove the Year column from our dataset.*** <br>

This decision stems from the observation that the Year column exhibits a low linear relationship with the Global_Sales column.

<h3>ANOVA (Analysis of Variance)</h3>
ANOVA can be used to determine if there are any statistically significant differences between the means of different groups of a categorical feature with respect to a continuous target variable.

In [12]:
from scipy import stats

anova_results = {}
for feat in categorical_feats:
    groups = df_sales.groupby(feat)["Global_Sales"].apply(list)
    f_val, p_val = stats.f_oneway(*groups)
    anova_results[feat] = {"F-value": f_val, "P-value": p_val}

df_anova = pd.DataFrame(anova_results).T

In [13]:
df_anova

,F-value,P-value
Name,2.813974,0.000000e+00
Platform,20.376542,2.224783e-107
Genre,18.176106,1.335889e-36
Publisher,3.380873,4.901851e-136


- Features with low P-values (typically < 0.05) are generally kept as they are likely to have a significant impact on the target variable;

- Features with high P-values (>= 0.05) may be candidates for removal, but consider the context and domain knowledge before making a final decision;

We decided to keep all columns except the Name, because of the low P-values.

<h3>Conclusion</h3>
At the end we want to initially remove two columns from the dataset (Year & Name).

In [14]:
df = df_sales.drop(columns=["Year", "Name"])

In [15]:
df

,Rank,Platform,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,1,Wii,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
1,2,NES,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
2,3,Wii,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
3,4,Wii,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
4,5,GB,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37
...,...,...,...,...,...,...,...,...,...
16593,16596,GBA,Platform,Kemco,0.01,0.00,0.00,0.00,0.01
16594,16597,GC,Shooter,Infogrames,0.01,0.00,0.00,0.00,0.01
16595,16598,PS2,Racing,Activision,0.00,0.00,0.00,0.00,0.01
16596,16599,DS,Puzzle,7G//AMES,0.00,0.01,0.00,0.00,0.01


<h3>Label Encoding</h3>
We drop the last column, because of the dummy trap.

In [16]:
for feat_name in categorical_feats:
    if (feat_name == "Name"): continue

    df = pd.get_dummies(df, columns=[feat_name])    
    df = df.drop(columns=[df.columns[-1]])

bool_cols = df.select_dtypes(include="bool").columns
df[bool_cols] = df[bool_cols].astype(int)

<h3>Splitting data into training and testing datasets</h3>

In [17]:
from sklearn.model_selection import train_test_split

X = df.iloc[:, df.columns != "Global_Sales"]
y = df.iloc[:, df.columns == "Global_Sales"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=23)

<h3>Feature Scaling</h3>

In [18]:
from sklearn.preprocessing import StandardScaler

continuous_feats_list = continuous_feats.tolist()
continuous_feats_list.remove("Year")
continuous_feats_list.remove("Global_Sales")

ss = StandardScaler()
X_train[continuous_feats_list] = ss.fit_transform(X_train[continuous_feats_list])
X_test[continuous_feats_list] = ss.transform(X_test[continuous_feats_list])

In [19]:
X_train

,Rank,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Platform_2600,Platform_3DO,Platform_3DS,Platform_DC,Platform_DS,...,Publisher_Zoo Games,Publisher_Zushi Games,Publisher_bitComposer Games,Publisher_dramatic create,Publisher_fonfun,Publisher_iWin,Publisher_id Software,Publisher_imageepoch Inc.,Publisher_inXile Entertainment,"Publisher_mixi, Inc"
15225,1.444474,-0.315298,-0.278639,-0.185678,-0.253484,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
14097,1.208272,-0.315298,-0.278639,-0.122519,-0.253484,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1920,-1.339322,0.469333,0.329680,-0.248837,0.271741,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
8755,0.090653,-0.146301,-0.259629,-0.248837,-0.200962,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3751,-0.956252,-0.001446,0.101560,-0.248837,0.114174,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11391,0.642140,-0.315298,-0.278639,0.003800,-0.253484,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
10370,0.428533,-0.254942,-0.202599,-0.248837,-0.200962,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9424,0.230617,-0.218728,-0.202599,-0.248837,-0.200962,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
13265,1.034206,-0.267013,-0.259629,-0.248837,-0.253484,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
X_test

,Rank,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Platform_2600,Platform_3DO,Platform_3DS,Platform_DC,Platform_DS,...,Publisher_Zoo Games,Publisher_Zushi Games,Publisher_bitComposer Games,Publisher_dramatic create,Publisher_fonfun,Publisher_iWin,Publisher_id Software,Publisher_imageepoch Inc.,Publisher_inXile Entertainment,"Publisher_mixi, Inc"
4264,-0.848925,0.143409,-0.259629,-0.248837,0.061651,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11995,0.768505,-0.242871,-0.259629,-0.248837,-0.200962,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2397,-1.239527,0.553831,-0.221609,-0.248837,0.376787,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
13749,1.135466,-0.303227,-0.240619,-0.248837,-0.253484,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
14034,1.195092,-0.315298,-0.278639,-0.122519,-0.253484,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9412,0.228107,-0.315298,-0.278639,0.161697,-0.253484,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6161,-0.452047,-0.315298,-0.278639,0.635391,-0.253484,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11006,0.561593,-0.291156,-0.183589,-0.248837,-0.148439,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2502,-1.217559,0.239979,0.310670,-0.248837,0.009129,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


<h3>Save data to CSV</h3>

In [21]:
pd.concat([X_train, y_train], axis=1).to_csv("./data/train.csv", index=False)
pd.concat([X_test, y_test], axis=1).to_csv("./data/test.csv", index=False)